In [1]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [1]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [2]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PINECONE_API_KEY)

print(pc.list_indexes())

[{
    "name": "churchill-speech",
    "metric": "cosine",
    "host": "churchill-speech-ppx2yn7.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}]


In [4]:
index_data = "churchill-speech"
index_info = pc.describe_index(index_data)
print(index_info)

{'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'churchill-speech-ppx2yn7.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'churchill-speech',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}


In [7]:
pc.list_indexes()

[
    {
        "name": "churchill-speech",
        "metric": "cosine",
        "host": "churchill-speech-ppx2yn7.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [11]:
index_data = 'churchill-speech'
if index_data in pc.list_indexes():
    print(index_data)
    pc.delete_index(index_data)
    print("Index deleted successfully.")
else:
    print(index_data)
    print("Index not found.")

churchill-speech
Index not found.


In [17]:
# Import the Pinecone library
from pinecone import ServerlessSpec

index_name = 'langchain'  # vector store / database table name

if index_name not in pc.list_indexes():
    print("Creating Index.")

    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec = ServerlessSpec(
            cloud ="aws",
            region="us-east-1"
        )
   )


    print("Index created successfully.")
else:
    print("Index already exists.")

Creating Index.
Index created successfully.


In [20]:
import random

vector = [[random.random() for _ in range(1536)] for _ in range(5)]

ids = list('abcde')

index = pc.Index(index_name)

index.upsert(vectors= zip(ids , vector))

{'upserted_count': 5}

In [22]:
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

In [24]:
index.fetch(ids=['a', 'b'])

FetchResponse(namespace='', vectors={'b': Vector(id='b', values=[0.157171547, 0.406052321, 0.330676913, 0.61082989, 0.47753334, 0.804587483, 0.566029608, 0.0954092667, 0.765535, 0.434170872, 0.148702621, 0.193335071, 0.0766607821, 0.53329134, 0.165020198, 0.345469624, 0.226748273, 0.201497674, 0.729243577, 0.99251, 0.109019883, 0.125799417, 0.878723562, 0.0950574279, 0.995866537, 0.90558821, 0.51034236, 0.906279624, 0.699391723, 0.408263087, 0.379102468, 0.0963094234, 0.621124804, 0.0174054746, 0.965957761, 0.184785515, 0.702414453, 0.42543003, 0.221352085, 0.464429945, 0.615908802, 0.44135341, 0.739239633, 0.558702826, 0.0371933691, 0.656335592, 0.694959521, 0.751541197, 0.85902679, 0.466187418, 0.921542048, 0.455089271, 0.317099601, 0.27529341, 0.399811953, 0.645795047, 0.367362589, 0.276722789, 0.798816562, 0.24258703, 0.513620436, 0.970891476, 0.918162525, 0.417142898, 0.645892084, 0.549767673, 0.297252983, 0.891619205, 0.360937715, 0.425956428, 0.067406252, 0.797098875, 0.56256544

In [26]:
index.delete(ids=['a', 'b'])

{}

In [27]:
import random

vector = [[random.random() for _ in range(1536)] for _ in range(5)]

ids = list('abfgh')

index = pc.Index(index_name)

index.upsert(vectors= zip(ids , vector))

{'upserted_count': 5}

In [30]:
import random

query_data = [random.random() for _ in range(1536)]

query_result = index.query(vector=query_data, top_k=3)

print(query_result)
#

{'matches': [{'id': 'c', 'score': 0.862890899, 'values': []},
             {'id': 'f', 'score': 0.751725435, 'values': []},
             {'id': 'e', 'score': 0.75166744, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}
